In [1]:
onGPU = True
atCaltech = False
onEOS = True
gpuFraction = 1.0

In [2]:
if onGPU:
    import setGPU

setGPU: Setting GPU to: 0


In [3]:
import numpy as np
# keras imports
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Conv2D, Dropout, Flatten, Concatenate, Reshape, BatchNormalization
from keras.layers import MaxPooling2D, MaxPooling3D
from keras.utils import plot_model
from keras import regularizers
from keras import backend as K
from keras import metrics
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from keras.regularizers import l1
# hyperparameters
import GPy, GPyOpt

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
if onGPU and gpuFraction<1.0:
    # limit GPU usage
    import sys
    import tensorflow as tf
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpuFraction)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
    import keras
    from keras import backend as K
    K.set_session(sess)
    print('using gpu memory fraction: '+str(gpuFraction))

In [5]:
# need afs password to read from eos
import os
import getpass
if onEOS: os.system("echo %s| kinit" %getpass.getpass())

········


In [7]:
%matplotlib inline

In [8]:
import h5py
import glob
import numpy as np

# Prepare train and test samples



In [9]:
target = np.array([])
jetImage = np.array([])
fileIN = "../data/jetImage_30.h5"
print(fileIN)
f = h5py.File(fileIN)
X = np.array(f.get("jetImage"))
Y = np.array(f.get('jets')[0:,-6:-1])
print(X.shape, Y.shape)

../data/jetImage_30.h5
(66000, 100, 100) (66000, 5)


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(44220, 100, 100) (21780, 100, 100) (44220, 5) (21780, 5)


In [12]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(44220, 100, 100, 1) (21780, 100, 100, 1) (44220, 5) (21780, 5)


In [13]:
n_epochs = 50
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
image_shape = (img_rows, img_cols, 1)

# Build the model

In [50]:
from keras.activations import relu, selu, elu
# myModel class
class myModel():
    def __init__(self, x_train, x_test, y_train, y_test, optmizer_index=0, CNN_filters=10, CNN_filter_size=5,
                 CNN_layers=2, CNN_activation_index=0, DNN_neurons=40, DNN_layers=2, DNN_activation_index=0, 
                 dropout=0.2, batch_size=100, epochs=50):
        self.activation = [relu, selu, elu]
        self.optimizer = ['adam', 'nadam','adadelta']
        self.optimizer_index = optmizer_index
        self.CNN_filters = CNN_filters
        self.CNN_filter_size = CNN_filter_size
        self.CNN_layers = CNN_layers
        self.CNN_activation_index = CNN_activation_index
        self.DNN_neurons = DNN_neurons
        self.DNN_layers = DNN_layers
        self.DNN_activation_index = DNN_activation_index
        self.dropout = dropout
        self.batch_size = batch_size
        self.epochs = epochs
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = x_train, x_test, y_train, y_test
        self.__model = self.build()
    
    #  model
    def build(self):
        inputImage = Input(shape=(image_shape))
        x = Conv2D(self.CNN_filters, kernel_size=(self.CNN_filter_size,self.CNN_filter_size), 
                   data_format="channels_last", strides=(1, 1), padding="same", 
                   activation= self.activation[self.CNN_activation_index], input_shape=image_shape,
                    kernel_initializer='lecun_uniform', name='cnn2D_0_relu')(inputImage)
        x = Dropout(self.dropout)(x)
        for i in range(1,self.CNN_layers):
            x = Conv2D(self.CNN_filters, kernel_size=(self.CNN_filter_size,self.CNN_filter_size), 
                   data_format="channels_last", strides=(1, 1), padding="same", 
                   activation= self.activation[self.CNN_activation_index], input_shape=image_shape,
                    kernel_initializer='lecun_uniform', name='cnn2D_%i_relu' %i)(x)
            x = Dropout(self.dropout)(x)
        ####
        x = Flatten()(x)
        #
        for i in range(self.DNN_layers):
            x = Dense(self.DNN_neurons, activation=self.activation[self.DNN_activation_index], 
                      kernel_initializer='lecun_uniform', name='dense_%i_relu' %i)(x)
            x = Dropout(self.dropout)(x)
        #
        output = Dense(5, activation='softmax', kernel_initializer='lecun_uniform', name = 'output_softmax')(x)
        ####
        model = Model(inputs=inputImage, outputs=output)
        model.compile(optimizer=self.optimizer[self.optimizer_index], loss='categorical_crossentropy', metrics=['acc'])
        return model

    
    # fit model
    def model_fit(self):
        self.__model.fit(self.__x_train, self.__y_train,
                        batch_size=self.batch_size,
                        epochs=self.epochs,
                        verbose=0,
                        validation_data=[self.__x_test, self.__y_test],
                        callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=0),
                                    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=0),
                                    TerminateOnNaN()])       
    
    # evaluate mnist model
    def model_evaluate(self):
        self.model_fit()
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation


In [51]:
# Runner function for model
# function to run mnist class

def run_model(x_train, x_test, y_train, y_test, optmizer_index=0, CNN_filters=10, CNN_filter_size=5,
                 CNN_layers=2, CNN_activation_index=0, DNN_neurons=40, DNN_layers=2, DNN_activation_index=0, 
                 dropout=0.2, batch_size=100, epochs=50):
    
    _model = myModel(x_train, x_test, y_train, y_test, optmizer_index, CNN_filters, CNN_filter_size,
                 CNN_layers, CNN_activation_index, DNN_neurons, DNN_layers, DNN_activation_index, 
                 dropout, batch_size, epochs)
    model_evaluation = _model.model_evaluate()
    return model_evaluation


# Bayesian Optimization

In [52]:
# the bounds dict should be in order of continuous type and then discrete type
bounds = [{'name': 'optmizer_index',        'type': 'discrete',  'domain': (0, 1, 2)},
          {'name': 'CNN_filters',           'type': 'discrete',  'domain': (5, 10, 15, 20, 25, 30)},
          {'name': 'CNN_filter_size',       'type': 'discrete',  'domain': (3, 5, 7, 9)},
          {'name': 'CNN_layers',            'type': 'discrete',    'domain': (1, 2, 3, 4, 5)},
          {'name': 'CNN_activation_index',  'type': 'discrete',    'domain': (0, 1, 2)},
          {'name': 'DNN_neurons',           'type': 'discrete',    'domain': (10, 20, 30, 40, 50, 60, 70, 80, 90, 100)},
          {'name': 'DNN_layers',            'type': 'discrete',    'domain': (1, 2, 3)},
          {'name': 'DNN_activation_index',  'type': 'discrete',    'domain': (0, 1, 2)},
          {'name': 'dropout',               'type': 'continuous',  'domain': (0.1, 0.4)},
          {'name': 'batch_size',            'type': 'discrete',    'domain': (50, 100, 200, 500)}]

In [53]:
# function to optimize model
def f(x):
    print(x)
    evaluation = run_model(X_train, X_test, Y_train, Y_test,
        optmizer_index = int(x[:,0]), 
        CNN_filters = int(x[:,1]), 
        CNN_filter_size = int(x[:,2]),
        CNN_layers = int(x[:,3]), 
        CNN_activation_index = int(x[:,4]), 
        DNN_neurons = int(x[:,5]), 
        DNN_layers = int(x[:,6]),
        DNN_activation_index = int(x[:,7]),
        dropout = float(x[:,8]),
        batch_size = int(x[:,9]),
        epochs = n_epochs)
    print("LOSS:\t{0} \t ACCURACY:\t{1}".format(evaluation[0], evaluation[1]))
    print(evaluation)
    return evaluation[0]

In [ ]:
# run optimization
opt_model = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)
opt_model.run_optimization(max_iter=10000)

[[0.00000000e+00 1.50000000e+01 3.00000000e+00 1.00000000e+00
  1.00000000e+00 3.00000000e+01 2.00000000e+00 1.00000000e+00
  1.00380733e-01 5.00000000e+02]]
LOSS:	0.7838268925335782 	 ACCURACY:	0.73627180676596
[0.7838268925335782, 0.73627180676596]
[[2.0000000e+00 1.0000000e+01 7.0000000e+00 4.0000000e+00 1.0000000e+00
  9.0000000e+01 1.0000000e+00 0.0000000e+00 3.9415882e-01 5.0000000e+02]]


# Result

In [ ]:
# print optimized model
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
\t{8}:\t{9}
\t{10}:\t{11}
\t{12}:\t{13}
""".format(bounds[0]["name"],opt_mnist.x_opt[0],
           bounds[1]["name"],opt_mnist.x_opt[1],
           bounds[2]["name"],opt_mnist.x_opt[2],
           bounds[3]["name"],opt_mnist.x_opt[3],
           bounds[4]["name"],opt_mnist.x_opt[4],
           bounds[5]["name"],opt_mnist.x_opt[5],
           bounds[6]["name"],opt_mnist.x_opt[6],
           bounds[7]["name"],opt_mnist.x_opt[7],
           bounds[8]["name"],opt_mnist.x_opt[8],
           bounds[9]["name"],opt_mnist.x_opt[9]))
print("optimized loss: {0}".format(opt_mnist.fx_opt))

In [ ]:
opt_mnist.x_opt

In [ ]:
outFile = open("OptCNN_30.log", "w")
# print optimized mnist model
outFile.write("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
\t{8}:\t{9}
\t{10}:\t{11}
\t{12}:\t{13}
""".format(bounds[0]["name"],opt_mnist.x_opt[0],
           bounds[1]["name"],opt_mnist.x_opt[1],
           bounds[2]["name"],opt_mnist.x_opt[2],
           bounds[3]["name"],opt_mnist.x_opt[3],
           bounds[4]["name"],opt_mnist.x_opt[4],
           bounds[5]["name"],opt_mnist.x_opt[5],
           bounds[6]["name"],opt_mnist.x_opt[6],
           bounds[7]["name"],opt_mnist.x_opt[7],
           bounds[8]["name"],opt_mnist.x_opt[8],
           bounds[9]["name"],opt_mnist.x_opt[9]))
outFile.write("\n")
outFile.write("optimized loss: {0}".format(opt_mnist.fx_opt))
outFile.close()